In [8]:
import sys
sys.path.extend(['../', '../read_utils/'])

import json
import numpy as np
from pathlib import Path

from read_utils.main import calibrate_magnetic_wifi_ibeacon_to_position, extract_wifi_rssi, visualize_heatmap, save_figure_to_html


In [7]:
floor_data_dir = '../data/site1/F1'
path_data_dir = floor_data_dir + '/path_data_files'
floor_plan_filename = floor_data_dir + '/floor_image.png'
floor_info_filename = floor_data_dir + '/floor_info.json'

save_dir = '../output/site1/F1'
wifi_image_save_dir = save_dir + '/wifi_images'

In [3]:
with open(floor_info_filename) as f:
    floor_info = json.load(f)
width_meter = floor_info["map_info"]["width"]
height_meter = floor_info["map_info"]["height"]
print(width_meter, height_meter)

path_filenames = list(Path(path_data_dir).resolve().glob("*.txt"))
mwi_datas = calibrate_magnetic_wifi_ibeacon_to_position(path_filenames)
step_positions = np.array(list(mwi_datas.keys()))
print(step_positions.shape)

239.81749314504376 176.44116534000818
(7840, 2)


In [4]:
# WiFi APs = WiFi access points
# RSS = Received Signal Strength / RSSI = Received Signal Strength Indicator
# BSSID = Basic Service Set Identifier 
wifi_rssi = extract_wifi_rssi(mwi_datas)
print(len(wifi_rssi))
print(list(wifi_rssi.keys())[:5])

2524
['06:74:9c:a7:a3:c0', '0a:74:9c:a7:a3:c0', '16:74:9c:a7:a3:c0', '0e:74:9c:a7:a3:c0', '1e:74:9c:a7:a3:c0']


In [13]:
target_wifi = '06:74:9c:a7:a3:c0'
heat_positions = np.array(list(wifi_rssi[target_wifi].keys()))
heat_values = np.array(list(wifi_rssi[target_wifi].values()))[:, 0]

fig = visualize_heatmap(heat_positions, heat_values, floor_plan_filename, width_meter, height_meter, colorbar_title='dBm', title=f'Wifi: {target_wifi} RSSI', show=False)
html_filename = f'{wifi_image_save_dir}/{target_wifi.replace(":", "-")}.html'
html_filename = str(Path(html_filename).resolve())
save_figure_to_html(fig, html_filename)

In [14]:
heat_positions.shape, heat_values.shape, step_positions.shape

((1549, 2), (1549,), (7840, 2))

In [15]:
# Identify the missing positions
missing_positions_mask = np.isin(step_positions, heat_positions).all(axis=1)

# Extract the missing positions
missing_positions = step_positions[~missing_positions_mask]

missing_positions.shape

(6291, 2)

In [16]:
from scipy.interpolate import Rbf

# Use Radial Basis Function interpolation to estimate missing values
rbf = Rbf(heat_positions[:, 0], heat_positions[:, 1], heat_values, function='linear')
interpolated_values = rbf(missing_positions[:, 0], missing_positions[:, 1])

interpolated_values.shape

/Users/hanchong/Documents/AI6128/AI6128-Project1/.venv/lib/python3.11/site-packages/scipy/interpolate/_rbf.py:268: LinAlgWarning:

Ill-conditioned matrix (rcond=1.90906e-20): result may not be accurate.



(6291,)

In [17]:
# Combine the known and interpolated values
all_positions = np.vstack([heat_positions, missing_positions])
all_values = np.hstack([heat_values, interpolated_values])

fig = visualize_heatmap(all_positions, all_values, floor_plan_filename, width_meter, height_meter, colorbar_title='dBm', title=f'Wifi: {target_wifi} RSSI', show=False)
html_filename = f'{wifi_image_save_dir}/{target_wifi.replace(":", "-")}-rbf.html'
html_filename = str(Path(html_filename).resolve())
save_figure_to_html(fig, html_filename)

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor

# Split the known data into a training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(heat_positions, heat_values, test_size=0.2, random_state=42)

# Define a range of k values to search over
k_values = list(range(1, 51))
errors = []

# Train and validate KNN for each k value
for k in k_values:
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    errors.append(mse)

# Find the optimal k value
optimal_k = k_values[np.argmin(errors)]
min_error = min(errors)

optimal_k, min_error

(15, 17.369304659498205)

In [19]:
# Use the optimal k value to interpolate the missing values
knn_optimal = KNeighborsRegressor(n_neighbors=optimal_k)
knn_optimal.fit(heat_positions, heat_values)
knn_optimal_interpolated_values = knn_optimal.predict(missing_positions)

# Combine the known and KNN-interpolated values for heatmap visualization
all_knn_values = np.hstack([heat_values, knn_optimal_interpolated_values])

In [20]:
fig = visualize_heatmap(all_positions, all_knn_values, floor_plan_filename, width_meter, height_meter, colorbar_title='dBm', title=f'Wifi: {target_wifi} RSSI', show=False)
html_filename = f'{wifi_image_save_dir}/{target_wifi.replace(":", "-")}-knn.html'
html_filename = str(Path(html_filename).resolve())
save_figure_to_html(fig, html_filename)

In [21]:
def inverse_distance_weighting(known_points, known_values, unknown_points, power=2):
    """
    Interpolate using Inverse Distance Weighting.
    """
    interpolated_values = []
    for point in unknown_points:
        # Compute distances to known_points
        distances = np.linalg.norm(known_points - point, axis=1)
        # Avoid divide by zero
        distances[distances == 0] = 1e-5
        # Compute weights
        weights = 1.0 / distances**power
        # Compute weighted average
        value = np.sum(weights * known_values) / np.sum(weights)
        interpolated_values.append(value)
    return np.array(interpolated_values)

# Interpolate using IDW
idw_interpolated_values = inverse_distance_weighting(heat_positions, heat_values, missing_positions)

idw_interpolated_values.shape

(6291,)

In [23]:
all_idw_values = np.hstack([heat_values, idw_interpolated_values])

fig = visualize_heatmap(all_positions, all_idw_values, floor_plan_filename, width_meter, height_meter, colorbar_title='dBm', title=f'Wifi: {target_wifi} RSSI', show=False)
html_filename = f'{wifi_image_save_dir}/{target_wifi.replace(":", "-")}-idw.html'
html_filename = str(Path(html_filename).resolve())
save_figure_to_html(fig, html_filename)

In [22]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# Define the Gaussian Processes kernel
kernel = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))

# Create and train the Gaussian Process regressor
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gp.fit(heat_positions, heat_values)

# Predict using Gaussian Processes
gp_interpolated_values, _ = gp.predict(missing_positions, return_std=True)

gp_interpolated_values.shape

/Users/hanchong/Documents/AI6128/AI6128-Project1/.venv/lib/python3.11/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning:

lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



(6291,)

In [24]:
all_gp_values = np.hstack([heat_values, gp_interpolated_values])

fig = visualize_heatmap(all_positions, all_idw_values, floor_plan_filename, width_meter, height_meter, colorbar_title='dBm', title=f'Wifi: {target_wifi} RSSI', show=False)
html_filename = f'{wifi_image_save_dir}/{target_wifi.replace(":", "-")}-gp.html'
html_filename = str(Path(html_filename).resolve())
save_figure_to_html(fig, html_filename)